In [1]:
from kv import FilesystemKV

In [9]:
xs = FilesystemKV[bytes]('here', extension='.txt')
for i in range(100):
  (await xs.insert(f'nested-{i%2}/{i}', f'Hello-{i}'.encode())).unsafe()

In [11]:
await xs.clear()

Right(value=None, tag='right')